## Подготовка данных

**Отключим** все предупреждения по инструкции

In [1]:
import warnings

warnings.filterwarnings("ignore")
import numpy as np
import pandas as pd

pd.set_option("display.max.columns", 100)
import pylab as plt

%matplotlib inline
import seaborn as sns
from matplotlib import pyplot as plt

plt.rcParams["figure.figsize"] = (10, 8)

**Считаем** наш файл с данными

In [2]:
df = pd.read_csv("data/charts.csv")

**Распечатаем** первые 5 записи

In [6]:
df.head(5)

,track_id,name,country,date,position,streams,artists,artist_genres,duration,explicit
0,7GMeRGZax4uDKgz6wALWQw,It Was Always You,br,2015-03-17,138,6584,['Maroon 5'],['pop'],239920,False
1,7GMeRGZax4uDKgz6wALWQw,It Was Always You,br,2015-03-18,132,6963,['Maroon 5'],['pop'],239920,False
2,7GMeRGZax4uDKgz6wALWQw,It Was Always You,br,2015-03-19,128,7049,['Maroon 5'],['pop'],239920,False
3,7GMeRGZax4uDKgz6wALWQw,It Was Always You,br,2015-03-20,152,6976,['Maroon 5'],['pop'],239920,False
4,7GMeRGZax4uDKgz6wALWQw,It Was Always You,br,2015-03-21,160,6156,['Maroon 5'],['pop'],239920,False


## Описание набора данных

In [7]:
print(df.shape)
print(df.columns.values)

(626475, 10)
['track_id' 'name' 'country' 'date' 'position' 'streams' 'artists'
 'artist_genres' 'duration' 'explicit']


In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 626475 entries, 0 to 626474
Data columns (total 10 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   track_id       626475 non-null  object
 1   name           626337 non-null  object
 2   country        626475 non-null  object
 3   date           626475 non-null  object
 4   position       626475 non-null  int64 
 5   streams        626475 non-null  object
 6   artists        626475 non-null  object
 7   artist_genres  626475 non-null  object
 8   duration       626475 non-null  int64 
 9   explicit       626475 non-null  bool  
dtypes: bool(1), int64(2), object(7)
memory usage: 43.6+ MB


|     Название      | Описание                                       |      Тип       |
|---                |:--                                             |:--             |
| **track_id**      | Уникальрный id в spotify                       | строчный       |
| **name**          | Название песни                                 | строчный       |
| **country**       | Код страны, в которой появлялась песня в чарте | строчный       |
| **date**          | Дата появление в чарте                         | датированный   |
| **position**      | Позиция в чарте                                | количественный |
| **streams**       | Количество stream-ов при появлении в чарте     | количественный |
| **artists**       | Авторы (исполнители) песни                     | массив строк   |
| **artist_genres** | Жанры, в которых исполнители выступают         | массив строк   |
| **duration**      | Длительность песни в (ms)                      | количественный |
| **explicit**      | Возрастное ограничение (есть/нет)              | бинарный       |

Целевая переменная **position** - Позиция в чарте (в зависимости от страны)

In [ ]:
df.

## Визуальный анализ